In [1]:
from acdesign.atmosphere import Atmosphere
from acdesign.performance.operating_point import OperatingPoint
from acdesign.airfoils.polar import UIUCPolar


clarky = UIUCPolar.local("CLARKYB")
rg15 = UIUCPolar.local("RG15C")
sa7038 = UIUCPolar.local("SA7038")
e472 = UIUCPolar.local("E472")


In [2]:
import plotly.express as px
from acdesign.performance.propulsion import Propeller, FactorMotor, ConstantPropeller
from acdesign.performance.aero import AircraftAero, WingAero, FuseAero
import numpy as np
import pandas as pd
import plotly.graph_objects as go


def solar_wing(nrows, ncols, section):
    b = nrows * 0.14 + 1
    C = ncols * 0.13 + 0.07
    S = b * C

    return WingAero(b, S, [section], [0, 1])


def solar_plane_aero(nrows, ncols, section):
    wing = solar_wing(nrows, ncols, section)
    fus_length = wing.b / 4
    return AircraftAero(
        wing,
        WingAero(wing.b * 0.2, wing.S * 0.2, [e472], [0, 1]),
        WingAero(wing.b * 0.1, wing.S * 0.1, [e472], [0, 1]),
        FuseAero(fus_length, 0.05),
        0.02,
        fus_length * 0.75,
    )


In [ ]:
nrows = 25
ncols = 2


cell_power = 3  # watts per solar cell
section = clarky


v = np.linspace(5, 30, 20)
mass = np.linspace(1, 20, 5)

vs, masses = np.meshgrid(v, mass)
vs, masses = vs.flatten(), masses.flatten()


def solar_wing_performance(nrows, ncols, section, drag_multiplier=1.12):
    wing = solar_wing(nrows, ncols, section)

    propeller = ConstantPropeller(0.5, 3000)
    motor = FactorMotor(0.65)
    atm = Atmosphere.alt(0)

    df = wing(atm, vs, masses*9.81)

    prop_results = propeller(df.drag * drag_multiplier, df.fs_v, atm.rho)
    motor_power = motor(prop_results.rpm, prop_results.torque)

    return pd.concat([df, prop_results], axis=1).assign(
        mass=masses,
        motor_power=motor_power, 
        solar_power=nrows * ncols * cell_power
    )


solar_wing_performance(30, 2, clarky)



,wing_l,fs_v,Cl,Cd0,Cm,k,Cd,lift,drag,moment,rpm,torque,mass,motor_power,solar_power
0,9.81,5.000000,970.703789,196.216698,170.014152,0.0202,19230.396770,25506.0,5.052937e+05,1474.194015,3000,1.801408e+04,1.0,8.706599e+06,180
1,9.81,6.315789,608.375118,19.574354,-10.627225,0.0202,7496.165130,25506.0,3.142752e+05,-147.029590,3000,1.415259e+04,1.0,6.840256e+06,180
2,9.81,7.631579,416.675467,7.784598,-35.167299,0.0202,3514.952726,25506.0,2.151612e+05,-710.390885,3000,1.170783e+04,1.0,5.658652e+06,180
3,9.81,8.947368,303.135007,5.805532,-38.658430,0.0202,1862.040327,25506.0,1.566734e+05,-1073.406978,3000,9.995138e+03,1.0,4.830870e+06,180
4,9.81,10.263158,230.390577,5.476194,-39.379847,0.0202,1077.711613,25506.0,1.193109e+05,-1438.684622,3000,8.730907e+03,1.0,4.219839e+06,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,196.20,24.736842,793.173536,63.145486,46.974980,0.0202,12771.729208,510120.0,8.213984e+06,9969.759414,3000,1.448759e+06,20.0,7.002172e+08,180
96,196.20,26.052632,715.078192,36.447026,13.314237,0.0202,10365.673278,510120.0,7.394628e+06,3134.361281,3000,1.373618e+06,20.0,6.638999e+08,180
97,196.20,27.368421,647.973498,21.805250,-6.744557,0.0202,8503.354955,510120.0,6.694304e+06,-1752.195269,3000,1.306331e+06,20.0,6.313785e+08,180
98,196.20,28.684211,589.889854,13.737224,-18.867114,0.0202,7042.883435,510120.0,6.090486e+06,-5384.195714,3000,1.245641e+06,20.0,6.020457e+08,180


In [7]:
results = solar_wing_performance(30, 2, clarky)

colors = px.colors.qualitative.Plotly

fig = go.Figure()
for i, (k, df) in enumerate(results.groupby("mass")):
    fig.add_trace(go.Scatter(
        x=df.fs_v,
        y=df.motor_power,
        mode='lines',
        name=f'Mass: {k:.1f} kg',
        line=dict(color=colors[i % len(colors)])
    ))


fig.update_layout(
    template="plotly_white",
    xaxis=dict(title='Airspeed (m/s)'),
    yaxis=dict(title='Power Required (Watt)'),
    margin=dict(l=0, r=0, b=0, t=0),
    width=800, height=600
)

In [5]:

def solar_performance(nrows, ncols, section):
    solar_power = nrows * ncols * cell_power

    aero = solar_plane_aero(nrows, ncols, clarky)

    propeller = ConstantPropeller(0.5, 3000)
    motor = FactorMotor(0.65)

    atm = Atmosphere.alt(0)

    performance = aero.trim(
        atm, np.linspace(5, 20, 50), np.linspace(5, 25, 20) * 9.81, npoints=1000
    )
    performance = pd.concat(
        [performance, propeller(performance.drag, performance.fs_v, atm.rho)], axis=1
    )
    performance = pd.concat(
        [performance, motor(performance.rpm, performance.torque)], axis=1
    )

    return performance.assign(
        solar_power=solar_power, nrows=nrows, ncols=ncols, section=section.name
    )


In [7]:
perf = solar_performance(30, 2, clarky)

In [30]:
fig = go.Figure()

for i, (k,df) in enumerate(perf.groupby(["in_lift"])):
    df = df.sort_values("fs_v")
    df = df.loc[df.power <= df.solar_power*2]
    fig.add_trace(go.Scatter(
        x=df.fs_v,
        y=df.power,
        name=f"{round(k[0]/9.81, 0)} kg",
        mode="lines"
    ))
fig.add_hline(y=perf.solar_power.iloc[0], line_dash="dash", 
              annotation_text="Solar Power Available", 
              annotation_position="bottom right")
fig.update_layout(
    template="plotly_white",
    xaxis=dict(title='Airspeed (m/s)'),
    yaxis=dict(title='Power Required (Watt)'),
    margin=dict(l=0, r=0, b=0, t=40),
    title="Performance for Solar Aircraft, 30 x 2 panel layout, Clarky",
    width=800, height=400
)


In [8]:
perf.iloc[0].T

(
    perf.loc[:, ["drag_fin", "drag_fuselage", "drag_tail"]].sum(axis=1)
    / perf.loc[:, "drag"]
).mean()

np.float64(0.13070055273865663)

In [ ]:
from joblib import Parallel, delayed
import os


def run_and_dump(nrow, ncol, section):
    perf = solar_performance(nrow, ncol, section)
    perf.to_csv(
        f"examples/solar_performance/solar_{nrow}x{ncol}_{section.name}.csv",
        index=False,
    )


#Parallel(n_jobs=os.cpu_count() * 2 - 4)(
#    delayed(run_and_dump)(nrows, ncols, clarky)
#    for nrows in range(25, 50, 3)
#    for ncols in range(1, 4)
#)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [9]:
from pathlib import Path
import pandas as pd
sizes = pd.concat(
    pd.read_csv(f) for f in Path("examples/solar_performance").glob("*.csv")
)

In [11]:
from tempfile import template
import plotly.graph_objects as go
can_fly = sizes.groupby(["nrows", "ncols"]).apply(
    lambda df: df.loc[(df.power < df.solar_power)], include_groups=False
)

can_fly = can_fly.reset_index()

fig = go.Figure()

for ncols in can_fly.ncols.unique():
    df = can_fly.loc[can_fly.ncols == ncols].groupby("nrows").max()

    fig.add_trace(
        go.Scatter(
            x=df.index, y=df.lift / 9.81, mode="lines", name=f"{ncols} chordwise panel"
        )
    )

fig.update_layout(
    template="plotly_white",
    margin=dict(l=0, r=0, t=40, b=0),
    xaxis=dict(title="Number of spanwise solar panals"),
    yaxis=dict(title="Mass (kg)"),
    title="Maximum mass for continous solar-powered flight at sea level",
    width=600, height=300
)


In [3]:
40 * 0.125

5.0

In [ ]:
# performance = solar_performance(25, 2, clarky)

max_weight_df = (
    performance.groupby("in_lift")
    .apply(lambda df: df[df.power <= df.solar_power].max(), include_groups=False)
    .dropna()
)
fig = go.Figure(
    data=[
        go.Scatter(
            x=max_weight_df.lift / 9.81,
            y=max_weight_df.fs_v,
            mode="lines",
            name="Max Weight",
        )
    ],
    layout=dict(
        title="Max Airspeeds for continuous Solar-Powered flight",
        xaxis_title="Weight (kg)",
        yaxis_title="Airspeed (m/s)",
    ),
)
fig

In [74]:
min_power_df = performance.groupby("in_lift").apply(
    lambda df: df.loc[df.power == df.power.min()]
)
min_power_df = min_power_df.assign(mass=min_power_df.in_lift / 9.81)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=min_power_df.mass,
        y=min_power_df.power,
        mode="lines",
        name="Minimum Power",
    )
)
fig.add_trace(
    go.Scatter(
        x=min_power_df.mass,
        y=min_power_df.fs_v,
        mode="lines",
        name="Minimum Power Airspeed",
        yaxis="y2",
    )
)

fig.add_hline(
    y=solar_power,
    line_dash="dash",
    annotation_text="Solar Power Available",
    annotation_position="top left",
)
fig.update_layout(
    title="Solar Plane Performance at Min Power Speed",
    xaxis_title="Mass (kg)",
    yaxis=dict(
        title="Power (W)",
    ),
    yaxis2=dict(title="Velocity (m/s)", overlaying="y", side="right"),
)

# px.line(performance.sort_values(by=["in_lift", "fs_v"]), x="fs_v", y="power", color="in_lift")

/tmp/ipykernel_3641061/352806568.py:1: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [68]:
heaviest_solar_plane(40, 3)

     message: `xtol` termination condition is satisfied.
     success: True
      status: 3
         fun: [ 1.838e-06]
           x: [ 2.487e+00  1.577e+01]
        cost: 1.6888960011635759e-12
         jac: [[ 1.598e+01 -7.135e+01]]
        grad: [ 2.937e-05 -1.311e-04]
  optimality: 0.0001311264931246031
 active_mask: [ 0.000e+00  0.000e+00]
        nfev: 38
        njev: 16

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
)
for ncols in df.ncols.unique():
    dff = df[df.ncols == ncols]
    fig.add_trace(
        go.Scatter(
            x=dff.nrows,
            y=dff.mass,
            mode="lines",
            line=dict(color=px.colors.qualitative.Plotly[ncols]),
            name=f"ncols={ncols}",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=dff.nrows,
            showlegend=False,
            y=dff.V,
            mode="lines",
            line=dict(color=px.colors.qualitative.Plotly[ncols]),
            name=f"ncols={ncols}",
        ),
        row=2,
        col=1,
    )

fig.update_layout(
    xaxis2=dict(title="n spanwise panels"),
    yaxis=dict(title="Max mass, kg"),
    yaxis2=dict(title="Optimal cruise speed, m/s"),
).show()

In [74]:
plane[0].trim(OperatingPoint(Atmosphere.alt(0), 11), 1.5 * 9.81)

,S,c,AR,Cl,Cd0,k,Cd,stall,Cm,re,p,gCl,gCd,gCd0
wing,1.716,0.33,15.757576,0.082191,0.011847,0.028858,0.012042,False,-0.090851,244322.560337,0.020,0.082191,0.012042,0.011847
tail,0.3432,0.264,4.924242,-0.167578,0.011991,0.092345,0.014584,False,0.003027,195458.048269,0.975,-0.033516,0.002917,0.002398
fin,0.1716,0.22,3.545455,0.0,0.009313,0.128257,0.009313,False,-0.005756,162881.706891,0.000,0.0,0.000931,0.000931
fuse,0.204204,1.3,NaN,0.0,0.004503,NaN,0.004503,NaN,0.0,962482.813448,0.000,0.0,0.000536,0.000536


In [ ]:
nrows = 30
ncols = 2
plane = (
    solar_plane_aero(nrows, ncols),
    FactorMotor(0.65),
    ConstantPropeller(0.5, 3000),
    Atmosphere.alt(0),
)


data = []
for mass in np.linspace(7, 16, 3):
    for V in np.linspace(5, 20, 20):
        data.append([mass, V, preq(*plane, mass, V)])


df = pd.DataFrame(data, columns=["mass", "V", "preq"])


fig = go.Figure()

for mass in df["mass"].unique():
    fig.add_trace(
        go.Scatter(
            x=df[df["mass"] == mass]["V"],
            y=df[df["mass"] == mass]["preq"],
            mode="lines",
            name=f"mass={mass:.2f} kg",
        )
    )

fig.update_xaxes(title="Airspeed (m/s)")
fig.update_yaxes(title="Power Required (W)", range=[0, 200])
fig.add_hline(
    y=cell_power * nrows * ncols,
    line_dash="dash",
    annotation_text="Solar Power Available",
    annotation_position="top left",
)
fig